![Banner](img/banner.png)

# **Taller:**  Descomposición de Valores Singulares (SVD) Análisis de Componentes Principales (PCA) 

**Semana 2 - Práctica Calificada -** Matrices simétricas positivas definidas

**Profesor:** *Fernando Lozano* - **Autor Notebook:** *César Garrido Urbano*


# Introducción

## Descripción


El presente *jupyter notebook* contine todo el material para el desarrollo de la prácitca calificada de la Semana 2 del curso ***Matématicas para Machine Learning***. 
En este se pondran en práctica algunas de las aplicaciones más conocidas relacionadas con algunos de los temas visto a lo largo del curso, especificamente el de *Singular Value Decomposotion* (SVD).

**Objetivos de Aprendizaje:**

*   Repasar los conceptos clave de la descomoposición de valores singulares (SVD).
*   Conocer las distintas aplicaciones que tiene la técnica de SVD.
*   Implementar algunas de estas aplicaciones en contextos prácticos como:

  *   Compresión de imagenes
  *   Reducción de dimensionalidad
  *   Clasificación de caracteristicas principales


## Metodología

En la primera parte del cuaderno usted encontrará varias implementaciones sencillas de SVD para algunos ejemplos prácticos. En esta primera mitad usted podrá experimentar con dichos ejemplos, deberá responder algunas preguntas a partir de los resultados y/o completar algunas funciones para la visualización de los datos. 

En la segunda parte del cuaderno pondrá en práctica lo estudiado y repasado en la primera mitad para un ejercicio en especifico. En este caso trabajando con un subconjunto de datos del bien conocido y estudiado dataset de [Perros vs Gatos](https://www.kaggle.com/competitions/dogs-vs-cats/data).

## Teoría


### Rango de una matriz

Recuerde que el rango de una matriz esta dado por el número de de filas (o columnas) linealmente independientes dentro de una matriz. Es decir filas que no se puedan expresar como una combinación lineal de otras filas:

\begin{equation}
r_3 \neq a r_1 + b r_2 
\end{equation}

En este caso $r_3$ es linealmente independiente de $r_1$ y $r_2$. De forma intuitiva, este valor esta asociado a la cantidad de información unica que nos brinda esta matriz, a mayor número de filas independientes, mayor la información única que nos da cada fila.

### SVD

Esto es importante pues la esencia de la descomposición de valores singulares (SVD) es la respresentación de una matriz $A$ como el producto de 3 matrices de la siguiente manera:

\begin{equation}
A_{m\times n} = U_{m\times m}S_{m\times n}V_{n\times n}^T = \begin{bmatrix}
            u_1 & ... & u_n
            \end{bmatrix}
            \begin{bmatrix}
            \sigma_1 &  & 0 \\
              & \ddots &  \\
             0 &  & \sigma_n
            \end{bmatrix}
            \begin{bmatrix}
            v_1^T \\ \vdots \\ v_n^T
            \end{bmatrix} = \sum_{i=1}^n \sigma_i u_i v_i^T
\end{equation}

Donde: 

*   $A$ es la matriz original
*   $U$ es una matriz mxm de vectores singulares izquierdos ($u_i$)
*   $S$ es una matriz diagonal con los valores singulares ($\sigma_i$)
*   $V$ es una matriz nxn con los vectores singulares derechos ($v_i$)



Esta descomposición, en últimas, nos permite respresentar nuestra matriz original ($A$) como una combinación lineal de matrices ($\sigma_i u_i v_i^T$) de rango 1!

Adicionalmente, es posible truncar dicha suma en un número $k$ inferior de matrices sin perder mucha información:





![TruncatedSVD.png](img/TruncatedSVD.png)

# 1. Implementación SVD


## Importar Datos

In [ ]:
# Librerias principales
import os
import cv2
import zipfile
import numpy as np
import pandas as pd

# Sklearn
from sklearn.datasets import load_digits, load_sample_images
from sklearn.decomposition import TruncatedSVD, PCA

# Tensorflow
import tensorflow as tf

# Visualización
import plotly.express as px
import matplotlib.pyplot as plt 

from IPython.display import display, HTML

In [ ]:
# Dataset de números escritos a mano
(X_digitos, y_digitos), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalizar imagenes entre [-1, 1]
X_digitos = (X_digitos - 127.5) / 127.5

# Tomar solo los primeros 1000 datos
X_digitos = X_digitos[0:1000]
y_digitos = y_digitos[0:1000]

In [ ]:
# Dataset de imagenes de ejemplo
dataset_img = load_sample_images()

## Imagenes Comprimidas

En esencia, las imagenes no son más que matrices dónde cada número denota la intensidad de cada píxel. Tome como ejemplo la siguiente imagen cargada desde la librería de $\texttt{sci-kit learn}$.

In [ ]:
# Visualizar imagen de ejemplo
img_ejemplo = dataset_img.images[0] 
plt.imshow(img_ejemplo)
plt.axis('off')
plt.title('Imagen de Ejemplo')
plt.show()

Inspecione las dimensiones y los valores de la imagen.

In [ ]:
img_ejemplo.shape

In [ ]:
img_ejemplo

Note que esta imagen esta compuesta por tres matrices de 427x640. Cada matriz esta asociada a la intensidad de tres colores: rojo, azul y verde (o RGB, por sus siglas en inglés), las cuales logran componer una imagen a color.

Dado que se desea trabajar con una única matriz, se hace la conversión a blanco y negro con la siguiente función:

In [ ]:
def convertir_blanco_y_negro(img):
    # Verificar tipo
    if type(img) != np.ndarray:
        img = np.array(img)
    
    # Verificar dimensiones (Imagen a color)
    assert img.shape[2] == 3, "Las dimensiones de la imagen no coinciden! Debe tener las tres dimensiones de color RGB"

    # Se convierte la imagen a blanco y negro (Se pasa de 3 matrices a 1 matriz)
    img_bn = 0.2125*img[:, :, 0] + 0.7154*img[:, :, 1] + 0.0721*img[:, :, 2]

    return img_bn

In [ ]:
# Se convierte la imagen a blanco y negro (Se pasa de 3 matrices a 1 matriz)
img_bn = convertir_blanco_y_negro(img_ejemplo)

Inspecione las nuevas dimensiones

In [ ]:
img_bn.shape

Ahora, con esta unica matriz procedemos a realizar la descomposicón por vectores singulares utilizando la librería de $\texttt{numpy}$.

In [ ]:
# Se realiza la descomposicion SVD
U, S, V = np.linalg.svd(img_bn)

Inspecciones las dimensiones y los valores de las matrices $U, S, V^T$.

In [ ]:
# Edite para imprimir valores o dimensiones
print(U.shape)
print(S)
print(V.shape)

Finalmente, se reconstruyen estas imagenes solo con un número determinado de componentes.

In [ ]:
# Numero de componentes de prueba
n_componentes = [3, 10, 25, 50, 200, 427]

# Figura de las diferentes imagenes
plt.figure(figsize = (16, 8))

for i, n in enumerate(n_componentes):
    # Reconstruccion de imagen con solo n componentes
    img_reducida = U[:, :n] @ np.diag(S[:n]) @ V[:n, :]
    # Gráfica
    plt.subplot(2, 3, i+1)
    plt.imshow(img_reducida, cmap = 'gray')
    plt.title(f'n_componentes = {n}')
    plt.axis('off')


¿Cuantas componentes son realmente necesarias para reconstruir la imagen?

Varie el número de componentes en el vector $\texttt{n componentes}$ para explorar distintos niveles de compresión de la imagen.

Ahora bien, para visualizar de mejor manera la información que aportan cada uno de los componentes a la reconstrucción de la imagen se puede graficar el aporte de cada componente ($S$) con la clase $\texttt{TruncatedSVD}$ de la librería de $\texttt{sci-kit learn}$.

In [ ]:
def graficar_valor_componentes(img):
    """
    Gráfica el aporte acumulado de cada componente dada una imagen en blanco y negro.
    ___________________________________
    Entrada:
    img:        [numpy.ndarray] Matriz de una imagen en blanco y negro
    ___________________________________
    Salida:
    fig         [plt.fig] Figura con gráfico
    """
    fig = plt.figure(figsize=(8, 5))
    
    # Varianza Truncated SVD
    svd = TruncatedSVD(n_components=426).fit(img_bn)
    var_svd = np.cumsum(svd.explained_variance_ratio_)

    # Grafica
    plt.plot(var_svd)
    plt.grid('on')
    
    # Títulos
    plt.xlabel('Número de Componentes')
    plt.ylabel('Varianza acumulada (%)')
    plt.title('Aporte porcentual acumulado por cada componenete')

    return fig

In [ ]:
# Probar función
graficar_valor_componentes(img_bn)
plt.show()

¿Coincide esta gráfica con las imagenes que se obtienen para cada número de componentes? Responda ahora sí ¿Cuantas componentes cree que son necesarias ara reconstruir la imagen?

**Respuesta:**

## Reducción de dimensionalidad

Considere este otro ejemplo del conocido dataset $\texttt{MNIST}$, el cual contiene imagenes (ya en blanco y negro) de digitos del 0 al 9 escritos a mano.

Inspecione las dimensiones y los valores de la imagen.

In [ ]:
# Imagen de ejemplo
digit_sample_img = X_digitos[0]
digit_sample_img = digit_sample_img.reshape((28, 28))

# Imprimir tamaño de la imagen
print(digit_sample_img.shape)

In [ ]:
# Visualizacion de las primeras 9 imagenes del dataset
fig = plt.figure(figsize=(9, 9))

# Plot
for i in range(0,9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_digitos[i].reshape((28, 28)), cmap = 'gray')
    plt.title(f'Digito {y_digitos[i]}')
    plt.axis('off')

plt.show()

Ahora se desea descomponer estas imagenes en tan solo 3 componentes para visualizar la posición de estos digitos en otro espacio (de tan solo 3 dimensiones!).

Para esto realizamos la descomposición con la clase $\texttt{TruncatedSVD}$ de la librería de $\texttt{sci-kit learn}$.

In [ ]:
# Descomposición en 3 dimensiones con Truncated SVD
svd = TruncatedSVD(n_components=3)
X_digits_transformed = svd.fit_transform(X_digitos.reshape(1000,28*28))

# Conversión a DF para su visualización
df = pd.DataFrame(X_digits_transformed)
df.columns = ['Componente 1', 'Componente 2', 'Componente 3']
df['Digito'] = y_digitos

# Visualización de datos
df.head(10)

In [ ]:
# Selección solo de los primeros 4 digitos (0, 1, 2, 3)
filtro = df['Digito'] < 4
df_filtrado = df[filtro]

# Plot (Gráfica en 3D)
fig = px.scatter_3d(df_filtrado, x='Componente 1', y='Componente 2', z='Componente 3', color='Digito')
fig.update_traces(marker=dict(size=6, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.show()

Ahora bien, la clase $\texttt{TruncatedSVD}$ solo implementa parte del algoritmo PCA (Principal Component Analysis) que ya fue implementado en previos laboratorios. Por suerte, la librería de $\texttt{sci-kit learn}$ también provee esta clase!

In [ ]:
# Descomposición en 3 dimensiones con PCA
pca = PCA(n_components=3)
X_digits_transformed = pca.fit_transform(X_digitos.reshape(1000,28*28))

# Conversión a DF para su visualización
df = pd.DataFrame(X_digits_transformed)
df.columns = ['Componente 1', 'Componente 2', 'Componente 3']
df['Digito'] = y_digitos

# Selección solo de los primeros 4 digitos (0, 1, 2, 3)
filtro = df['Digito'] < 4
df_filtrado = df[filtro]

# Plot (Gráfica en 3D)
fig = px.scatter_3d(df_filtrado, x='Componente 1', y='Componente 2', z='Componente 3', color='Digito')
fig.update_traces(marker=dict(size=6, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.show()

¿Nota alguna relación entre la posición de los digitos en el nuevo espacio 3D y la forma de los mismos?

Pruebe con distintas combinaciones de digitos, la implementación de SVD y PCA y escriba sus conclusiones a continuación.

**Respuesta:**


# 2. Ejercicio Práctico

## Importar Datos

In [ ]:
def cargar_imagenes(carpeta, dim=(64,64), byn = True):
    """
    Cargar imagenes de la carpeta por parametro.
    ___________________________________
    Entrada:
    carpeta:        [numpy.ndarray] Matriz de una imagen en blanco y negro.
    dim:            [tuple] Dimensiones a las que se quiere estandarizar las imagenes.
    byn:            [boolean] Si se desea o no convertir las imagenes a blanco y negro.
    ___________________________________
    Salida:
    imagenes        [list(numpy.ndarray)] Arreglo de imagenes.
    """
    imagenes = []
    # Recorrer todos los archivos en la carpeta
    for archivo in os.listdir(carpeta):  
        # Verificar formato
        if archivo.endswith('.jpg'):
            # Cargar Imagen
            img = cv2.imread(os.path.join(carpeta, archivo))
            if img is None:
                pass
            # Corregir Color
            im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Ajustar a un único tamaño
            if dim is not None:
                img_ajustada = cv2.resize(im_rgb, dim, interpolation = cv2.INTER_AREA)
            # Convertir a blanco y negtro
            if byn:
                img_ajustada = convertir_blanco_y_negro(img_ajustada)
            # Agregar
            imagenes.append(img_ajustada)

    return np.array(imagenes)

In [ ]:
# Cargar imagenes de Gatos a color
img_gatos = cargar_imagenes("img/Gatos", byn = False)

# Visualizacion de las primeras 9 imagenes del dataset de Gatos
fig = plt.figure(figsize=(9, 9))

# Plot
for i, example in enumerate(img_gatos):
    plt.subplot(3, 3, i+1)
    plt.imshow(example, cmap = 'gray')
    plt.title(f"Gato {i}")
    plt.axis('off')
    if i >= 8:
        break

plt.show()

In [ ]:
# Cargar imagenes de Perros a color
img_perros = cargar_imagenes("img/Perros", byn = False)

# Visualizacion de las primeras 9 imagenes del dataset de Gatos
fig = plt.figure(figsize=(9, 9))

# Plot
for i, example in enumerate(img_perros):
    plt.subplot(3, 3, i+1)
    plt.imshow(example, cmap = 'gray')
    plt.title(f"Perro {i}")
    plt.axis('off')
    if i >= 8:
        break

plt.show()

## Compresión de Imagenes

Seleccione una imagen del subconjunto de perros y otro del de gatos y comprimala para distinto número de componentes tal y como se hizo para la imagen de ejemplo. ¿Cuantas componentes son necesarias para reconstruir la imagen del perro y del gato? ¿Esto varía dependiendo del ánimal? ¿De la foto?

In [ ]:
# Compersión imagen de gato


In [ ]:
# Compersión imagen de perro


**Respuesta:**

## Reducción de dimensionalidad

Descomponga todas las imagenes de los perros y los gatos y reduzcalos a tan solo 3 componentes. Visualice la ubicación de las imagenes en el nuevo espacio. ¿Tienen sentido? ¿Por qué sí o por qué no?

In [ ]:
# Reducción de dimensionalidad


In [ ]:
# Visualización


**Respuesta:**

## Proyección de los datos en un nuevo espacio

Por último, transforme las imagenes de perros y gatos por separado y proyecte algun ejemplo de su gusto en el espacio contrario. Es decir, tome una foto de un gato y proyectela en el espacio proyectado de los perros y viceversa. Gráfique la imagen resultante ¿Qué nota de interesante en estas nuevas imagenes?

Hint: Considere la función $\texttt{inverse_transform}$ de las clases $\texttt{PCA}$ y $\texttt{TruncatedSVD}$.

In [ ]:
# Proyección de gato en el espacio de perros


In [ ]:
# Proyección de perro en el espacio de gatos


**Respuesta:**